In [1]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Embedding, Flatten, Input, concatenate
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import keras.backend as K
import numpy as np
import pandas as pd
import pickle
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

def zuhe(line):
    line['hour'] = line['hour'].astype(str)
    line['weekday'] = line['weekday'].astype(str)
    line['is_peek'] = line['is_peek'].astype(str)
    line['is_workday'] = line['is_workday'].astype(str)
                                   
    line['hour_weekday'] = line['hour'] + line['weekday']
    line['peak_workday'] = line['is_peek'] + line['is_workday']
    
    line['hour'] = line['hour'].astype(int)
    line['is_peek'] = line['is_peek'].astype(int)
    line['is_workday'] = line['is_workday'].astype(int)
    line['weekday'] = line['weekday'].astype(int)
    
    line['hour_weekday'] = line['hour_weekday'].astype(int)
    line['peak_workday'] = line['peak_workday'].astype(int)
    return line

with open('./data/train-id4-crowd-grid4.txt', 'rb') as data_file:
    train = pickle.load(data_file)
    #训练验证都划分
    #900 29 26 #800 27.9 25.9 #700 27.5 25.3 #27.4 24.8 #26.4 24.08 #23 21 #(90)250 21 19.5 #(80)17. 16.5
    #训练划分 验证不划分
    #250
train = train[(train['Diff_Time']<600)]
train = zuhe(train)
with open('./data/test-id4-crowd-grid4.txt', 'rb') as data_file:
    test = pickle.load(data_file)
test = zuhe(test)
from sklearn import preprocessing
x1 = train['ID'].as_matrix().reshape([-1,1])
N1 = x1.shape[0]

x2 = test['ID'].as_matrix().reshape([-1,1])
N2 = x2.shape[0]
x = np.concatenate((x1,x2))
x = preprocessing.LabelEncoder().fit_transform(x) #13963746x21567
print(x.shape)#(14454126, 21567)
train['new_ID'] = x[:N1]
test['new_ID'] = x[N1:]

col1 = ['new_ID']
col2 = [c for c in train if
       c not in ['Unnamed: 0','ID2', 's_ij', 'e_ij', 's_x', 's_y', 'e_x', 'e_y', 'new_ID','ID','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station', 'O_TIME', 'aver_v', 'max_v',
                 'Diff_Time']]
col3 = ['Source_Station_encode']
col4 = ['Target_Station_encode']

X_train1 = train[col1].values
X_train2 = train[col2].values
X_train3 = train[col3].values
X_train4 = train[col4].values
y_train = train['Diff_Time'].values

print(col1,col2,col3,col4)
print('train',X_train1.shape, X_train2.shape, X_train3.shape, X_train4.shape)

col1 = ['new_ID']
col2 = [c for c in test if
       c not in ['Unnamed: 0','ID2', 's_ij', 'e_ij', 's_x', 's_y', 'e_x', 'e_y', 'new_ID','ID','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station', 'O_TIME', 'aver_v', 'max_v',
                 'Diff_Time','Distance1', 'distance2','TERMINALNO', 'new_dist']]
col3 = ['Source_Station_encode']
col4 = ['Target_Station_encode']

print(col1,col2,col3,col4)
X_test1 = test[col1].values
X_test2 = test[col2].values
X_test3 = test[col3].values
X_test4 = test[col4].values
print('test',X_test1.shape,X_test2.shape,X_test3.shape,X_test4.shape)

/home/kawayi-4/anaconda3/envs/python36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/kawayi-4/anaconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:43: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/kawayi-4/anaconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:46: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/kawayi-4/anaconda3/envs/python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or

(14223933,)
['new_ID'] ['Distance', 'hour', 'is_peek', 'weekday', 'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'h_aver_diff', 'h_aver_d', 'h_aver_v', 'is_crowd', 'grid_aver_diff', 'grid_aver_d', 'Source_Station_encode', 'Target_Station_encode', 'hour_weekday', 'peak_workday'] ['Source_Station_encode'] ['Target_Station_encode']
train (13733553, 1) (13733553, 20) (13733553, 1) (13733553, 1)
['new_ID'] ['Distance', 'hour', 'is_peek', 'weekday', 'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'h_aver_diff', 'h_aver_d', 'h_aver_v', 'is_crowd', 'grid_aver_diff', 'grid_aver_d', 'Source_Station_encode', 'Target_Station_encode', 'hour_weekday', 'peak_workday'] ['Source_Station_encode'] ['Target_Station_encode']
test (490380, 1) (490380, 20) (490380, 1) (490380, 1)


In [2]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)+1e-15, axis=-1))
earlyStopping=EarlyStopping(monitor='val_loss', patience=4, verbose=0, mode='auto')

ckpt_path = './log/weights-{val_loss:.4f}.hdf5'
ckpt = ModelCheckpoint(ckpt_path,
                       monitor='val_loss', 
                       verbose=1, 
                       save_best_only=True, 
                       mode='min')

In [3]:
from keras.layers import Reshape, Conv2D, MaxPooling2D
from keras.layers import Input, TimeDistributed, Dense, Lambda, concatenate, Dropout, BatchNormalization
from keras.optimizers import Nadam
# col1 = ['new_ID']
# col2 = [c for c in train if
#        c not in ['new_ID','ID','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station', 'O_TIME', 'aver_v', 'max_v',
#                  'Diff_Time']]

input1 = Input(shape=(1,))
input2 = Input(shape=(20,))
input3 = Input(shape=(1,))
input4 = Input(shape=(1,))

x1 = Embedding(21064, 128, input_length=1)(input1)
x1 = Flatten()(x1)
x1 = BatchNormalization()(x1)
x2 = Dense(units=128)(input2)
x2 = BatchNormalization()(x2)
x2 = Activation("relu")(x2)
x3 = Embedding(4609, 64, input_length=1)(input3)
x3 = BatchNormalization()(x3)
x3 = Flatten()(x3)
x4 = Embedding(4609, 64, input_length=1)(input4)
x4 = BatchNormalization()(x4)
x4 = Flatten()(x4)

def dist(x):
    return x[0]*x[1]/(K.sum(x[0]**2,axis=1,keepdims=True)+K.sum(x[1]**2,axis=1,keepdims=True))    
#     Aggregate
x3_x4 = Lambda(dist)([x3,x4])

x = concatenate([x1, x2, x3, x4, x3_x4])
# #CNN
# x = Reshape((20, 20, 1))(x)
# x = Conv2D(32, (3, 3), padding='same')(x)
# x = MaxPooling2D(pool_size=(2, 2))(x)
# x = Conv2D(16, (3, 3), padding='same')(x)
# x = MaxPooling2D(pool_size=(2, 2))(x)
# x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
out = Dense(1, activation=None)(x)
model = Model(inputs=[input1, input2, input3, input4], outputs=out)
print(model.summary())
# model.load_weights('./log/weights-21.7671.hdf5')
model.compile(loss=root_mean_squared_error, optimizer=Nadam(), metrics=[root_mean_squared_error])
model.fit([X_train1,X_train2, X_train3, X_train4], y_train, validation_split=0.1, epochs=10, batch_size=1024,callbacks=[ckpt,earlyStopping])
# model.save("keras3.h5")

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_

12360197/12360197 [==============================] - 351s 28us/step - loss: 23.8943 - root_mean_squared_error: 23.8943 - val_loss: 23.6050 - val_root_mean_squared_error: 23.6050

Epoch 00009: val_loss did not improve from 23.19506
Epoch 10/10
12360197/12360197 [==============================] - 354s 29us/step - loss: 23.8183 - root_mean_squared_error: 23.8183 - val_loss: 23.0669 - val_root_mean_squared_error: 23.0669

Epoch 00010: val_loss improved from 23.19506 to 23.06687, saving model to ./log/weights-23.0669.hdf5


In [5]:
# with open('./data/test5-id.txt', 'rb') as data_file:
#     test = pickle.load(data_file)
# test = zuhe(test)
col1 = ['new_ID']
col2 = [c for c in test if
       c not in ['new_ID','ID','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station', 'O_TIME', 'aver_v', 'max_v',
                 'Diff_Time','Distance1', 'distance2','TERMINALNO', 'new_dist']]
print(col1,col2)
X_test1 = test[col1].values
X_test2 = test[col2].values
print(X_test1.shape,X_test2.shape)

model.load_weights('./log/weights-21.7671.hdf5') #(21.7671==<0.03789)
y_pred1 =  model.predict([X_test1,X_test2],batch_size=1024)[:,0]

# test['pred'] = y_pred1
# sub1 = test[['O_LINENO','TERMINALNO', 'O_UP','Source_Station','Target_Station','O_TIME','pred','Distance']]
# sub1.columns = ['LINE','TERMINALNO','UP','pred_start_stop_ID','pred_end_stop_ID','realTime','pred','Distance']
# sub1 = sub1.reset_index()
# del sub1['index']

# sub=pd.read_csv("./toBePredicted_0605_segment.csv", sep=",")
# sub['realTime'] = pd.to_datetime(sub['realTime'],format='%Y-%m-%d %H:%M:%S')
# sub2 = sub[['LINE','TERMINALNO','UP','pred_start_stop_ID','pred_end_stop_ID','realTime','distance']]
# sub2=pd.merge(sub2,sub1,on=['LINE','TERMINALNO','UP','pred_start_stop_ID','pred_end_stop_ID','realTime'],how='left')
# sub2
# sub2.to_csv('./toBePredicted_06010_resultx.csv',sep=",",index=False)#0605 29.2467

print('finished')

['new_ID'] ['Distance', 'hour', 'is_peek', 'weekday', 'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'h_aver_diff', 'h_aver_d', 'h_aver_v', 'hour_weekday', 'peak_workday']
(490380, 1) (490380, 15)
finished


In [6]:
# with open('./data/test5-id.txt', 'rb') as data_file:
#     test = pickle.load(data_file)
# test = zuhe(test)
test = test[['new_dist', 'ID', 'O_LINENO', 'O_UP', 'Source_Station', 'Target_Station',
       'Distance', 'distance2', 'O_TIME', 'hour', 'is_peek', 'weekday',
       'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'max_v',
       'h_aver_diff', 'h_aver_d', 'h_aver_v', 'TERMINALNO', 
       'hour_weekday']]
test.columns=['Distance', 'ID', 'O_LINENO', 'O_UP', 'Source_Station', 'Target_Station',
       'Distance1', 'distance2', 'O_TIME', 'hour', 'is_peek', 'weekday',
       'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'max_v',
       'h_aver_diff', 'h_aver_d', 'h_aver_v', 'TERMINALNO', 
       'hour_weekday']
# col1 = [c for c in test if
#        c not in ['ID','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station', 'O_TIME', 'aver_v', 'max_v',
#                  'Diff_Time','Distance1', 'distance2','TERMINALNO']]
# print(col1)

# X_test = test[col1].values
# print(X_test.shape)

# model.load_weights('./log/weights-23.5826.hdf5') 
# y_pred2 =  model.predict(X_test,batch_size=1024)[:,0]

In [7]:
test['pred1'] = y_pred1
test['pred2'] = y_pred1

sub1 = test[['O_LINENO','TERMINALNO', 'O_UP','Source_Station','Target_Station','O_TIME','pred1','pred2','Distance','Distance1']]
sub1.columns = ['LINE','TERMINALNO','UP','pred_start_stop_ID','pred_end_stop_ID','realTime','pred1','pred2','Distance','Distance1']
sub1 = sub1.reset_index()
del sub1['index']

sub=pd.read_csv("./toBePredicted_0607_segment.csv", sep=",")
sub['realTime'] = pd.to_datetime(sub['realTime'],format='%Y-%m-%d %H:%M:%S')
sub2 = sub[['LINE','TERMINALNO','UP','pred_start_stop_ID','pred_end_stop_ID','realTime','distance']]
sub2=pd.merge(sub2,sub1,on=['LINE','TERMINALNO','UP','pred_start_stop_ID','pred_end_stop_ID','realTime'],how='left')
sub2
# sub2.to_csv('./toBePredicted_0607_resultx.csv',sep=",",index=False)#0605 29.2467

# print('finished')

,LINE,TERMINALNO,UP,pred_start_stop_ID,pred_end_stop_ID,realTime,distance,pred1,pred2,Distance,Distance1
0,678,904388,0,11,12,2017-10-28 14:00:00,0.417984,123.647491,123.647491,0.679068,1.097053
1,678,904388,0,12,13,2017-10-28 14:00:00,0.000000,142.524384,142.524384,0.744740,0.744740
2,678,904388,0,13,14,2017-10-28 14:00:00,0.000000,62.777367,62.777367,0.479191,0.479191
3,678,904388,0,14,15,2017-10-28 14:00:00,0.000000,82.143997,82.143997,0.367919,0.367919
4,678,904388,0,15,16,2017-10-28 14:00:00,0.000000,246.684952,246.684952,1.141722,1.141722
5,678,904388,0,16,17,2017-10-28 14:00:00,0.000000,120.675827,120.675827,0.578620,0.578620
6,678,904388,0,17,18,2017-10-28 14:00:00,0.000000,76.187889,76.187889,0.410412,0.410412
7,678,904388,0,18,19,2017-10-28 14:00:00,0.000000,109.688072,109.688072,0.665032,0.665032
8,678,904388,0,19,20,2017-10-28 14:00:00,0.000000,169.106354,169.106354,0.907266,0.907266
9,678,904388,0,20,21,2017-10-28 14:00:00,0.000000,129.756317,129.756317,0.471062,0.471062


In [174]:
np.percentile(a,97)

['Distance', 'hour', 'is_peek', 'weekday', 'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'h_aver_diff', 'h_aver_d', 'h_aver_v', 'hour_weekday']
(490380, 14)


In [23]:
sub2[(sub2['new_pred'].isnull())]

,LINE,TERMINALNO,UP,pred_start_stop_ID,pred_end_stop_ID,realTime,distance,pred1,pred2,Distance,Distance1,div_dist,new_pred


In [152]:
# a = []
# for i in range(sub2.shape[0]):
#     l = sub2.iloc[i]
#     if l['distance'] > 10 or l['distance'] == -1:
#         a.append(l['Distance'])
#     elif l['Distance'] - l['distance'] < 0:
#         a.append(0)
#     else:
#         a.append(l['Distance'] - l['distance'])
# sub2['new_dist'] = a

In [8]:
sub2['div_dist'] = sub2['Distance'] / sub2['Distance1']
sub2['new_pred'] = sub2['div_dist'] * sub2['pred1']

In [9]:
import math
for i in range(sub2.shape[0]):
    s = sub2.iloc[i]
    if math.isnan(s['div_dist']):
        sub2.loc[i,'new_pred'] = s['pred1']


In [10]:
#计算用newpred和直接用新距离之间的偏差
print((sub2['pred1']-sub2['pred2']).mean())
print((sub2['pred2']-sub2['new_pred']).mean())
print((sub2['pred1']-sub2['new_pred']).mean())


0.0
4.351322014693011
4.351322014693011


In [12]:
sub2.to_csv('./toBePredicted_0612_resultn.csv',sep=",",index=False)#0605 29.2467

print('finished')

finished


In [11]:
sub2['new_pred'].mean()

127.102320290998

In [261]:
with open('./data/test5-id.txt', 'rb') as data_file:
    test = pickle.load(data_file)
test

,ID,O_LINENO,O_UP,Source_Station,Target_Station,Distance,distance2,O_TIME,hour,is_peek,...,nws,dts,nts,is_rain,max_v,h_aver_diff,h_aver_d,h_aver_v,TERMINALNO,new_dist
209,86600809,866,0,8,9,0.629479,0.269546,2017-10-25 09:00:00,9,0,...,1,17,10,0,-1,104.605948,0.623653,21.462932,905578,0.359933
210,86600910,866,0,9,10,0.517842,0.000000,2017-10-25 09:00:00,9,0,...,1,17,10,0,-1,121.091912,0.520073,15.461508,905578,0.517842
211,86601011,866,0,10,11,0.536539,0.000000,2017-10-25 09:00:00,9,0,...,1,17,10,0,-1,116.723636,0.535451,16.514425,905578,0.536539
212,86601112,866,0,11,12,0.776565,0.000000,2017-10-25 09:00:00,9,0,...,1,17,10,0,-1,169.844203,0.770747,16.336670,905578,0.776565
213,86601213,866,0,12,13,0.408048,0.000000,2017-10-25 09:00:00,9,0,...,1,17,10,0,-1,71.837545,0.403532,20.222203,905578,0.408048
214,86601314,866,0,13,14,0.983655,0.000000,2017-10-25 09:00:00,9,0,...,1,17,10,0,-1,152.003663,0.979596,23.200388,905578,0.983655
215,86601415,866,0,14,15,0.586326,0.000000,2017-10-25 09:00:00,9,0,...,1,17,10,0,-1,116.649635,0.589106,18.180779,905578,0.586326
216,86601516,866,0,15,16,0.480220,0.000000,2017-10-25 09:00:00,9,0,...,1,17,10,0,-1,76.299639,0.476011,22.459331,905578,0.480220
217,86601617,866,0,16,17,0.520341,0.000000,2017-10-25 09:00:00,9,0,...,1,17,10,0,-1,196.512635,0.525341,9.623949,905578,0.520341
218,86601718,866,0,17,18,0.794807,0.000000,2017-10-25 09:00:00,9,0,...,1,17,10,0,-1,193.818182,0.787386,14.624997,905578,0.794807


In [266]:
 test[(test['h_aver_v']==-1)] #Distance new_dist 440 #h_aver_diff 系列 1296

,ID,O_LINENO,O_UP,Source_Station,Target_Station,Distance,distance2,O_TIME,hour,is_peek,...,nws,dts,nts,is_rain,max_v,h_aver_diff,h_aver_d,h_aver_v,TERMINALNO,new_dist
5299,57402829,574,0,28,29,1.390789,0.000000,2017-10-25 07:00:00,7,1,...,1,17,10,0,-1,-1.0,-1.0,-1.0,901164,1.390789
7735,90501314,905,0,13,14,0.764686,0.000000,2017-10-25 17:00:00,17,1,...,1,17,10,0,-1,-1.0,-1.0,-1.0,901672,0.764686
23088,16301011,163,0,10,11,NaN,0.000000,2017-10-25 07:00:00,7,1,...,1,17,10,0,-1,-1.0,-1.0,-1.0,931279,NaN
25074,16301011,163,0,10,11,NaN,0.000000,2017-10-25 15:00:00,15,0,...,1,17,10,0,-1,-1.0,-1.0,-1.0,931279,NaN
30885,58811920,588,1,19,20,8.784893,10.957189,2017-10-25 15:00:00,15,0,...,1,17,10,0,-1,-1.0,-1.0,-1.0,904490,8.784893
36037,65101415,651,0,14,15,3.024193,0.000000,2017-10-25 13:00:00,13,0,...,1,17,10,0,-1,-1.0,-1.0,-1.0,902942,3.024193
48964,46210809,462,1,8,9,3.103658,0.000000,2017-10-25 13:00:00,13,0,...,1,17,10,0,-1,-1.0,-1.0,-1.0,901234,3.103658
51701,46201516,462,0,15,16,4.167935,0.000000,2017-10-25 07:00:00,7,1,...,1,17,10,0,-1,-1.0,-1.0,-1.0,901231,4.167935
56099,5501301213,55013,0,12,13,NaN,-1.000000,2017-10-25 09:00:00,9,0,...,1,17,10,0,-1,-1.0,-1.0,-1.0,905343,NaN
56100,5501301314,55013,0,13,14,NaN,0.000000,2017-10-25 09:00:00,9,0,...,1,17,10,0,-1,-1.0,-1.0,-1.0,905343,NaN


In [2]:
import numpy as np
import pandas as pd
import pickle
import os
# train2 = pd.read_csv("./train_id_loc_with_crowd.csv", sep=",")
# train2

In [10]:
with open('./data/test-id3-crowd-grid2.txt', 'rb') as data_file:
    line = pickle.load(data_file)
line

,Unnamed: 0,ID,O_LINENO,O_UP,Source_Station,Target_Station,Distance,distance2,O_TIME,hour,...,TERMINALNO,new_dist,s_x,s_y,e_x,e_y,is_crowd,s_ij,e_ij,ID2
0,209,86600809,866,0,8,9,0.629479,0.269546,2017-10-25 09:00:00,9,...,905578,0.359933,39.08480,117.1790,39.08030,117.1820,0,503,503,86605030503
1,210,86600910,866,0,9,10,0.517842,0.000000,2017-10-25 09:00:00,9,...,905578,0.517842,39.08030,117.1820,39.08062,117.1858,0,503,503,86605030503
2,211,86601011,866,0,10,11,0.536539,0.000000,2017-10-25 09:00:00,9,...,905578,0.536539,39.08062,117.1858,39.07970,117.1900,0,503,503,86605030503
3,212,86601112,866,0,11,12,0.776565,0.000000,2017-10-25 09:00:00,9,...,905578,0.776565,39.07970,117.1900,39.08070,117.1960,0,503,503,86605030503
4,213,86601213,866,0,12,13,0.408048,0.000000,2017-10-25 09:00:00,9,...,905578,0.408048,39.08070,117.1960,39.08320,117.1938,0,503,503,86605030503
5,214,86601314,866,0,13,14,0.983655,0.000000,2017-10-25 09:00:00,9,...,905578,0.983655,39.08320,117.1938,39.08860,117.2000,1,503,503,86605030503
6,215,86601415,866,0,14,15,0.586326,0.000000,2017-10-25 09:00:00,9,...,905578,0.586326,39.08860,117.2000,39.09032,117.2056,2,503,504,86605030504
7,216,86601516,866,0,15,16,0.480220,0.000000,2017-10-25 09:00:00,9,...,905578,0.480220,39.09032,117.2056,39.09140,117.2100,1,504,504,86605040504
8,217,86601617,866,0,16,17,0.520341,0.000000,2017-10-25 09:00:00,9,...,905578,0.520341,39.09140,117.2100,39.09600,117.2100,2,504,537,86605040537
9,218,86601718,866,0,17,18,0.794807,0.000000,2017-10-25 09:00:00,9,...,905578,0.794807,39.09600,117.2100,39.10280,117.2090,1,537,537,86605370537


In [13]:
def idfeature2(a,c,d):
    if c == -1:
        c = 0
    if d == -1:
        d = 0
    if c < 10:
        c = '000'+str(c)
    elif c < 100:
        c = '00'+str(c)
    elif c < 1000:
        c = '0'+str(c)
    if d < 10:
        d = '000'+str(d)
    elif d < 100:
        d = '00'+str(d)
    elif d < 1000:
        d = '0'+str(d)
    return str(a)+str(c)+str(d)
line['ID2'] = line.apply(lambda x: idfeature2(x['O_LINENO'],x['s_ij'],x['e_ij']),axis=1)
line

,Unnamed: 0,ID,O_LINENO,O_UP,Source_Station,Target_Station,Distance,distance2,O_TIME,hour,...,TERMINALNO,new_dist,s_x,s_y,e_x,e_y,is_crowd,s_ij,e_ij,ID2
0,209,86600809,866,0,8,9,0.629479,0.269546,2017-10-25 09:00:00,9,...,905578,0.359933,39.08480,117.1790,39.08030,117.1820,0,503,503,86605030503
1,210,86600910,866,0,9,10,0.517842,0.000000,2017-10-25 09:00:00,9,...,905578,0.517842,39.08030,117.1820,39.08062,117.1858,0,503,503,86605030503
2,211,86601011,866,0,10,11,0.536539,0.000000,2017-10-25 09:00:00,9,...,905578,0.536539,39.08062,117.1858,39.07970,117.1900,0,503,503,86605030503
3,212,86601112,866,0,11,12,0.776565,0.000000,2017-10-25 09:00:00,9,...,905578,0.776565,39.07970,117.1900,39.08070,117.1960,0,503,503,86605030503
4,213,86601213,866,0,12,13,0.408048,0.000000,2017-10-25 09:00:00,9,...,905578,0.408048,39.08070,117.1960,39.08320,117.1938,0,503,503,86605030503
5,214,86601314,866,0,13,14,0.983655,0.000000,2017-10-25 09:00:00,9,...,905578,0.983655,39.08320,117.1938,39.08860,117.2000,1,503,503,86605030503
6,215,86601415,866,0,14,15,0.586326,0.000000,2017-10-25 09:00:00,9,...,905578,0.586326,39.08860,117.2000,39.09032,117.2056,2,503,504,86605030504
7,216,86601516,866,0,15,16,0.480220,0.000000,2017-10-25 09:00:00,9,...,905578,0.480220,39.09032,117.2056,39.09140,117.2100,1,504,504,86605040504
8,217,86601617,866,0,16,17,0.520341,0.000000,2017-10-25 09:00:00,9,...,905578,0.520341,39.09140,117.2100,39.09600,117.2100,2,504,537,86605040537
9,218,86601718,866,0,17,18,0.794807,0.000000,2017-10-25 09:00:00,9,...,905578,0.794807,39.09600,117.2100,39.10280,117.2090,1,537,537,86605370537


In [14]:
with open('./data/test-id4-crowd-grid2.txt', 'wb') as data_file:
    pickle.dump(line, data_file)

In [13]:
line[(line['s_ij']==-1)|(line['e_ij']==-1)]

,Unnamed: 0,ID,O_LINENO,O_UP,Source_Station,Target_Station,Distance,distance2,O_TIME,hour,...,TERMINALNO,new_dist,s_x,s_y,e_x,e_y,is_crowd,s_ij,e_ij,ID2


In [12]:
def idfeature3(c):
    if c == -1:
        c = 0
    return c
line['s_ij'] = line.apply(lambda x: idfeature2(x['s_ij']),axis=1)
line['e_ij'] = line.apply(lambda x: idfeature2(x['e_ij']),axis=1)
